## Ejemplo de práctica que usa las características de programación orientada a objetos

En una universidad se requiere un programa para gestionar la información de los profesores. El programa debe permitir:
a)	Ingresar información de tantos profesores como quiera el usuario
b)	Calcular el pago total de toda la nómina de profesores de asignatura
c)	Imprimir los datos de todos los profesores

Existen dos categorías de profesores: de asignatura(A) y de planta(P). A los dos tipos de profesores se les debe asignar un id y capturar su nombre, grado y categoría. Esta información deberá registrarse en un archivo de texto validando la entrada del grado (LIC/MAE/DOC) y de la categoría (A/P).

Para los profesores de asignatura se debe también capturar el área en donde dan clase y el máximo de horas que se le puede asignar por ciclo. Hay cuatro funciones que deben poder realizarse a los profesores de asignatura por ciclo: 


1.	Pagar lo de un ciclo
2.	Asignarle una materia en un ciclo
3.	Calcular las horas totales que actualmente tiene asignadas en un ciclo
4.	Imprimir todos sus datos

Tanto los pagos como las materias asignadas deben de registrarse en los archivos de texto respectivos.
Para los profesores de planta, a su vez, se debe capturar la facultad donde trabaja y su salario base. Las funciones que se deben permitir para los profesores de planta son:


1.	Pagar a un profesor tomando en cuenta posibles gratificaciones
2.	Imprimir todos sus datos


<img src="Clases Profesor.png" style="width:500px;height:500px" />

### Clase abstracta de Profesor

In [20]:
from abc import ABC,  abstractmethod

class Profesor(ABC):   #ejemplo abstracción
    def __init__(self,idP,nom,gr,cat):
        self.idP=idP
        self.nombre=nom
        self.grado=gr
        self.categoria=cat #solo asignatura o planta
        self.guardaProf(idP,nom,gr,cat) #guarda un profesor en archivo profesores
        
    def guardaProf(self,idP,nom,gr,cat):
        with open('profesores.txt','a') as archprofs:
            linea=idP+','+nom+','+gr+','+cat+'\n'
            archprofs.write(linea)  
    
    @abstractmethod 
    def pago():
        pass
    
    def imprime(self): #Ejemplo de Polimorfismo
         #imprime datos generales del profesor 
        print('ID: ',self.idP)
        print('Nombre:',self.nombre)
        print('Grado: ',self.grado) 
        print('Categoria: ',self.categoria)

### Clase Asignatura 
Clase derivada de Profesor

In [23]:
class Asignatura(Profesor):     #ejemplo de herencia
    def __init__(self,idP,nom,gr,cat):
        super().__init__(idP,nom,gr,cat)
        #Ejemplo de encapsulamiento
        self.__area=input("En que area trabaja: ")
        self.__maxHrs=int(input("Max de hrs que puede impartir: "))
         
    def pago(self,ciclo):
        hrsaPagar=self.horasxCiclo(ciclo)
        if self.grado=='lic':
            tarifa=50
        elif self.grado=='mae':
            tarifa=80
        else:
            tarifa=100
        apagar=tarifa*hrsaPagar   
        #escribe el pago en un archivo
        with open('pagos.txt','a') as archpagos:  
            linea=idP+','+ciclo+','+str(apagar)+'\n'
            archpagos.write(linea) 
        return apagar
        
    def asignaClase(self):
        ciclo=input("Ciclo: ")
        materia=input("Materia: ")
        numHoras=int(input("Numero de horas de la clase: "))
        total=self.horasxCiclo(ciclo)   #revisa en el archivo cuantas horas tiene programadas para un ciclo
        if total+numHoras<=self.__maxHrs:
            #asigna una clase para un ciclo
            with open('materias.txt','a') as archmaterias:   
                linea=idP+','+ciclo+','+materia+','+str(numHoras)+'\n'
                archmaterias.write(linea) 
        else:
            print('Ya tiene '+str(total)+' asignadas')
            print('Imposible asignarle '+str(numHoras)+' mas')
      
    def horasxCiclo(self,ciclo):
        total=0
        try:
            #calcula cuantas horas tiene programadas
            with open('materias.txt') as archmaterias:
                while True:
                    linea = archmaterias.readline()
                    if not linea: break
                    lista=linea.strip().split(sep=',')
                    if lista[0]==self.idP and lista[1]==ciclo :
                         total=int(lista[3])+total
        except FileNotFoundError:
            print('no hay materias aun')
        return total
     
    def imprime(self):   #Ejemplo de polimorfismo
        #imprime datos generales del profesor 
        super().imprime()
        #imprime los datos de profesor de asignatura que faltan
        print('Area: ',self.__area)
        print('Sueldo base: ',self.__maxHrs)
        #extrae del archivo materias cuales tiene programadas el profesor y las imprime
        tiene=False
        try:
            print("Materias programadas ")
            with open('materias.txt') as archmaterias:
                 while True:
                    linea = archmaterias.readline()
                    if not linea: break
                    lista=linea.strip().split(sep=',')
                    if lista[0]==self.idP:
                        tiene=True
                        print(linea,end='')
        except:
             print('no hay materias programadas')    
        finally:
             if not tiene:
                print('El profesor no tiene materias programadas')   

### Clase Planta
Clase derivada de Profesor

In [24]:
class Planta(Profesor):   #ejemplo de herencia
    def __init__(self,idP,nom,gr,cat):
        super().__init__(idP,nom,gr,cat)
        #Ejemplo de encapsulamiento
        self.__facultad=input("En que facultad trabaja: ")
        self.__sueldo=float(input("Sueldo base: "))
         
    def pago(self):
        gratificacion=0
        tieneGrat=input('¿El profesor tiene gratificaciones (s/n)?')
        if tieneGrat=='s':
            gratificacion=float(input('Gratificacion: '))     
        return self.__sueldo+gratificacion
    
    def imprime(self):   #Ejemplo de polimorfismo
        #imprime datos generales del profesor 
        super().imprime() 
        #imprime los datos de profesor de planta que faltan
        print('Facultad: ',self.__facultad)
        print('Sueldo base: ',self.__sueldo)

### Menú para funciones de profesores de asignatura

In [25]:
def menuAsignatura(profAsig):
    opcion=1       
    while opcion!=5:
        print('\n1. Asigna una clase')
        print('2. Calcula cuantas horas tiene programadas')
        print('3. Imprime información del profesor')
        print('4. Calula pago del profesor' )
        print('5. Terminar\n')
        opcion=int(input('Da la opcion: '))
        if opcion==1: #Asigna una clase
            profAsig.asignaClase() 
        elif opcion==2:
            ciclo=input("Ciclo: ")
            print('Tiene '+str(profAsig.horasxCiclo(ciclo))+' programadas')
        elif opcion==3: 
            profAsig.imprime()
        elif opcion==4: 
            ciclo=input("Ciclo: ")
            print('Pago total: ',profAsig.pago(ciclo))
        elif opcion==5:
            print('Adios')
        else:
            print('Opcion invalida ')

### Menú para funciones de profesores de planta

In [26]:
def menuPlanta(profPlanta):
    opcion=1       
    while opcion!=3:
        print('\n1. Imprime información del profesor')
        print('2. Calula pago del profesor' )
        print('3. Terminar\n')
        opcion=int(input('Da la opcion: '))
        if opcion==1: #Imprime información del profesor
            profPlanta.imprime()
        elif opcion==2:
            print('Pago: ',profPlanta.pago())
        elif opcion==3:
            print('Adios')
        else:
            print('Opcion invalida ')   

### Aquí inicia el programa 

In [27]:
resp='s'
#a) ingresa tantos profesores como quiere el usuario y llama al menú de asignatura y de planta
while resp=='s':
    idP=input("ID: ")
    nombre=input("Nombre: ")
    grado=''
    while grado!='LIC' and grado!='MAE' and grado!='DOC': #valida el grado
        grado=input("Grado (LIC/MAE/DOC) : ").upper()  
    categoria=''
    while categoria!='A' and categoria!='P':  #valida la categoría
        categoria=input("Categoria (A asignatura P planta): ").upper() 
    if categoria=='A':  
        profAsig=Asignatura(idP,nombre,grado,categoria)
        menuAsignatura(profAsig) 
    else:
        profPlanta=Planta(idP,nombre,grado,categoria)
        menuPlanta(profPlanta)
    resp=input('¿Quieres dar de alta otro profesor? (s/n): ')  

ID: 15
Nombre: Bill Gates
Grado (LIC/MAE/DOC) : lic
Categoria (A asignatura P planta): a
En que area trabaja: ing
Max de hrs que puede impartir: 15

1. Asigna una clase
2. Calcula cuantas horas tiene programadas
3. Imprime información del profesor
4. Calula pago del profesor
5. Terminar

Da la opcion: 1
Ciclo: 1198
Materia: ada
Numero de horas de la clase: 10
no hay materias aun

1. Asigna una clase
2. Calcula cuantas horas tiene programadas
3. Imprime información del profesor
4. Calula pago del profesor
5. Terminar

Da la opcion: 2
Ciclo: 1198
Tiene 10 programadas

1. Asigna una clase
2. Calcula cuantas horas tiene programadas
3. Imprime información del profesor
4. Calula pago del profesor
5. Terminar

Da la opcion: 3
ID:  15
Nombre: Bill Gates
Grado:  LIC
Categoria:  A
Area:  ing
Sueldo base:  15
Materias programadas 
15,1198,ada,10

1. Asigna una clase
2. Calcula cuantas horas tiene programadas
3. Imprime información del profesor
4. Calula pago del profesor
5. Terminar

Da la opcion

In [28]:
#b) Pago de toda la nomina de profesores de asignatura
pagoTodos=0
try:
    with open('pagos.txt') as archpagos:
        while True:
            linea = archpagos.readline()
            if not linea: break
            lista=linea.strip().split(sep=',')
            pagoTodos=pagoTodos+int(lista[2])
except:
    print('no hay pagos aun')
finally:
    print('Pago total: ',pagoTodos)

Pago total:  1000


In [29]:
#c) imprime los datos de todos los profesores    
try:
    with open('profesores.txt') as archprofs:   
        print(archprofs.read())
except:
    print('no hay profesores')

1,Uno,lic,A
2,Dos,mae,P
3,Tres,doc,P
4,Cuatro,doc,A
5,Cinco,doc,A
6,Seis,doc,A
7,Siete,mae,A
8,ocho,DOC,A
8,ocho,DOC,A
8,ocho,LIC,A
9,nueve,MAE,P
9,nueve,LIC,P
9,nueve,LIC,P
10,diez,MAE,P
11,once,DOC,P
15,Bill Gates,LIC,A



### Archivos

<img src="Archivos.png" style="width:800px;height:300px" />